In [1]:
import pandas as pd
import datetime as dt
import os
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_auth
import plotly.graph_objects as go
import tab1
import tab2

# Wczytywanie danych

###### Stworzymy osobną klasę db, w której będą znajdować się podstawowe operacje na plikach źródłowych.

In [2]:
class db:
    def __init__(self):
        self.transactions = db.transaction_init()
        self.cc = pd.read_csv(r'db\country_codes.csv',index_col=0)
        self.customers = pd.read_csv(r'db\customers.csv',index_col=0)
        self.prod_info = pd.read_csv(r'db\prod_cat_info.csv')

###### Korzystając z biblioteki os, wczytujemy wszystkie pliki z folderu db\transactions do Pandas i łączymy je ze sobą dzięki funkcji append

In [3]:
@staticmethod
def transaction_init():
    transactions = pd.DataFrame()
    src = r'db\transactions'
    for filename in os.listdir(src):
        transactions = transactions.append(pd.read_csv(os.path.join(src,filename),index_col=0))

    def convert_dates(x):
        try:
            return dt.datetime.strptime(x,'%d-%m-%Y')
        except:
            return dt.datetime.strptime(x,'%d/%m/%Y')

    transactions['tran_date'] = transactions['tran_date'].apply(lambda x: convert_dates(x))

    return transactions

###### Chcemy również połączyć bazę z transakcjami z odpowiednimi kategoriami produktów, płcią klienta oraz krajem sprzedaży.

In [4]:
def merge(self):
    df = self.transactions.join(self.prod_info.drop_duplicates(subset=['prod_cat_code'])
    .set_index('prod_cat_code')['prod_cat'],on='prod_cat_code',how='left')

    df = df.join(self.prod_info.drop_duplicates(subset=['prod_subcat_code'])
    .set_index('prod_subcat_code')['prod_subcat'],on='prod_subcat_code',how='left')

    df = df.join(self.customers.join(self.cc,on='country_code')
    .set_index('customer_Id'),on='cust_id')

    self.merged = df
    
    df = db()
    df.merge()

# Budowa podstawowego layoutu

###### Inicjujemy aplikację, wykorzystując dodatkowo zewnętrzny arkusz stylów.

In [5]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

###### Widget umożliwiający wykorzystywanie kolejnych zakładek, tj. tabów. Będziemy go przechowywać wewnątrz divów, tak aby w dość prosty sposób kontrolować jego wysokość i szerokość.

In [6]:
app.layout = html.Div([html.Div([dcc.Tabs(id='tabs',value='tab-1',children=[
                            dcc.Tab(label='Sprzedaż globalna',value='tab-1'),
                            dcc.Tab(label='Produkty',value='tab-2')
                            ]),
                            html.Div(id='tabs-content')
                    ],style={'width':'80%','margin':'auto'})],
                    style={'height':'100%'})

# Sprzedaż globalna

In [7]:
def render_tab(df):

    layout = html.Div([html.H1('Sprzedaż globalna',style={'text-align':'center'}),
                        html.Div([dcc.DatePickerRange(id='sales-range',
                        start_date=df['tran_date'].min(),
                        end_date=df['tran_date'].max(),
                        display_format='YYYY-MM-DD')],style={'width':'100%','text-align':'center'}),
                        html.Div([html.Div([dcc.Graph(id='bar-sales')],style={'width':'50%'}),
                        html.Div([dcc.Graph(id='choropleth-sales')],style={'width':'50%'})],style={'display':'flex'})
                        ])

    return layout

# Produkty

In [8]:
def render_tab(df):

    grouped = df[df['total_amt']>0].groupby('prod_cat')['total_amt'].sum()
    fig = go.Figure(data=[go.Pie(labels=grouped.index,values=grouped.values)],layout=go.Layout(title='Udział grup produktów w sprzedaży'))

    layout = html.Div([html.H1('Produkty',style={'text-align':'center'}),

                        html.Div([html.Div([dcc.Graph(id='pie-prod-cat',figure=fig)],style={'width':'50%'}),
                        html.Div([dcc.Dropdown(id='prod_dropdown',
                                    options=[{'label':prod_cat,'value':prod_cat} for prod_cat in df['prod_cat'].unique()],
                                    value=df['prod_cat'].unique()[0]),
                                    dcc.Graph(id='barh-prod-subcat')],style={'width':'50%'})],style={'display':'flex'}),
                                    html.Div(id='temp-out')
                        ])

    return layout

# Callbacks

In [9]:
@app.callback(Output('tabs-content','children'),[Input('tabs','value')])
def render_content(tab):

    if tab == 'tab-1':
        return tab1.render_tab(df.merged)
    elif tab == 'tab-2':
        return tab2.render_tab(df.merged)

In [10]:
## tab1 callbacks
@app.callback(Output('bar-sales', 'figure'), [Input('sales-range', 'start_date'), Input('sales-range', 'end_date')])

SyntaxError: incomplete input (3720463816.py, line 2)

In [11]:
def tab1_bar_sales(start_date,end_date):

    truncated = df.merged[(df.merged['tran_date']>=start_date)&(df.merged['tran_date']<=end_date)]
    grouped = truncated[truncated['total_amt']>0].groupby([pd.Grouper(key='tran_date',freq='M'),'Store_type'])['total_amt'].sum().round(2).unstack()

    traces = []
    for col in grouped.columns:
        traces.append(go.Bar(x=grouped.index,y=grouped[col],name=col,hoverinfo='text',
        hovertext=[f'{y/1e3:.2f}k' for y in grouped[col].values]))

    data = traces
    fig = go.Figure(data=data,layout=go.Layout(title='Przychody',barmode='stack',legend=dict(x=0,y=-0.5)))

    return fig

In [12]:
@app.callback(Output('choropleth-sales','figure'),
            [Input('sales-range','start_date'),Input('sales-range','end_date')])
def tab1_choropleth_sales(start_date,end_date):

    truncated = df.merged[(df.merged['tran_date']>=start_date)&(df.merged['tran_date']<=end_date)]
    grouped = truncated[truncated['total_amt']>0].groupby('country')['total_amt'].sum().round(2)

    trace0 = go.Choropleth(colorscale='Viridis',reversescale=True,
                            locations=grouped.index,locationmode='country names',
                            z = grouped.values, colorbar=dict(title='Sales'))
    data = [trace0]
    fig = go.Figure(data=data,layout=go.Layout(title='Mapa',geo=dict(showframe=False,projection={'type':'natural earth'})))

    return fig

In [13]:
## tab2 callbacks
@app.callback(Output('barh-prod-subcat','figure'),
            [Input('prod_dropdown','value')])
def tab2_barh_prod_subcat(chosen_cat):

    grouped = df.merged[(df.merged['total_amt']>0)&(df.merged['prod_cat']==chosen_cat)].pivot_table(index='prod_subcat',columns='Gender',values='total_amt',aggfunc='sum').assign(_sum=lambda x: x['F']+x['M']).sort_values(by='_sum').round(2)

    traces = []
    for col in ['F','M']:
        traces.append(go.Bar(x=grouped[col],y=grouped.index,orientation='h',name=col))

    data = traces
    fig = go.Figure(data=data,layout=go.Layout(barmode='stack',margin={'t':20,}))
    return fig

# Uruchamianie

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False, port=8051)

Dash is running on http://127.0.0.1:8051/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8051
Press CTRL+C to quit
127.0.0.1 - - [09/Jul/2023 22:28:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 22:28:23] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_10_2m1687157202.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 22:28:23] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_10_2m1687157202.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 22:28:23] "GET /_dash-component-suites/dash/deps/react@16.v2_10_2m1687157202.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 22:28:23] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_10_2m1687157202.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 22:28:23] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_10_0m1687157202.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 22:28:23] "GET /_dash-component-suites/dash/dash_table/bundle.v5_2_5m1687157202.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 22:28:23] "GET /_dash-component-suites/dash/html/da